In [6]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from datetime import datetime
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import *
from quantfreedom.simulate import run_df_backtest, or_backtest
from quantfreedom.strategies.strategy import Strategy
from quantfreedom.strategies.ex_strat_01 import RSIBelowAbove

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
candles = mufex_main.get_candles(
    symbol="BTCUSDT",
    timeframe="5m",
    since_datetime=datetime(2023, 10, 15),
    until_datetime=datetime(2023, 10, 24),
)

In [3]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)

In [4]:
backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2.5]),
    sl_based_on_add_pct=np.array([0.05, 0.1, 0.15]),
    sl_based_on_lookback=np.array([20]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5]),
    trail_sl_when_pct=np.array([1]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

In [7]:
strategy = RSIBelowAbove(
    rsi_is_below=np.array([60]),
    rsi_length=np.array([14]),
    long_short="long",
)

In [10]:
run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
    logger_bool=True,
)

AttributeError: 'NoneType' object has no attribute 'market_fee_pct'

In [ ]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
    dos_index=1,
    ind_set_index=0,
    plot_results=True,
)

In [ ]:
order_records_df